In [1]:
import sys

sys.path.append('/Users/felix/Library/Mobile Documents/com~apple~CloudDocs/PhD/Python Libraries')
import pysimtra.src.pysimtra as ps

#### Import the SIMTRA executable into the package

In [2]:
# Define the path to the SIMTRA application folder
sim_path = '/Users/felix/Library/Mobile Documents/com~apple~CloudDocs/PhD/Python Libraries/sputtering_simulation/simtra_simulation/simtra_app'
# Import the application into the package
ps.import_exe(sim_path)

#### Set up a simple one-cathode sputter system programmatically

In [4]:
# Create a cylindrical sputter chamber
ch = ps.Chamber.cylindrical(radius=0.12, length=0.18, temperature=293, pressure=0.5)
# Construct the magnetron with a 2 inch target from a few simple surfaces
target = ps.Circle(name='target', radius=0.0191, position=(0, 0, 0.055))
shield = ps.Cone(name='shield', small_rho=0.0191, big_rho=0.023, height=0.005, position=(0, 0, 0.055))
cap = ps.Circle(name='cap', radius=0.03, position=(0, 0, 0.06))
cap.perforate(by='circle', radius=0.023)
body = ps.Cylinder(name='body', radius=0.03, height=0.06)
# Combine all surfaces into a dummy object 
m_object = ps.DummyObject(name='magnetron', surfaces=[target, shield, cap, body])
# Define a path to the racetrack file
r_path = '/Users/felix/Library/Mobile Documents/com~apple~CloudDocs/PhD/Python Libraries/sputtering_simulation/racetrack_database/racetracks_simtra/1.5inch_target_Ni_#5_EcoChem_24-05-02_thickness_1.27mm_simtra.txt'
# Finally construct the magnetron
mag = ps.Magnetron(transported_element='Au', 
                   m_object=m_object, 
                   n_particles=10**8, 
                   sputter_surface_index=1,  # the first surface is the target
                   racetrack_file_path=r_path,
                   max_ion_energy=200)
# Create a rectangle to track the particles arrival
s_surf = ps.Rectangle(name='subSurface', 
                      dx=0.05, dy=0.05, 
                      save_avg_data=True, 
                      avg_grid=(10, 10))
# Wrap the rectangle into a dummy object
sub = ps.DummyObject(name='substrate', surfaces=[s_surf], position=(0, 0, 0.15))
# Define an output path for the simulation
out_path = '/Users/felix/Downloads'
# Combine chamber, magnetron and substrate into a sputter system
system = ps.SputterSystem(ch, mag, sub, out_path)


In [ ]:
# Save the Sputter System as a sin file
system.to_sin(path='/Users/felix/Downloads/res.sin')

Perform the simulation. This part only works on Windows operating systems.

In [ ]:
# Start the simulation
results = system.simulate()

In [ ]:
# Plot the results


In [ ]:
d = pd.read_csv('/Users/felix/Library/Mobile Documents/com~apple~CloudDocs/PhD/Python Libraries/sputtering_simulation/periodic_table.csv')
d = d[['symbol', 'surface_binding_energy']]
d.to_csv('/Users/felix/Downloads/table.csv', index=False)